# Root Finding

### scipy.optimize.fsolve

The first tool is the `scipy.optimize` function `fsolve`. It takes two arguments:
- a function `f`
- a point `x` near which to search for a root of `f`.

In [14]:
import numpy as np
from scipy.optimize import fsolve
f= lambda x : np.cos(x)-x
print('root for f is',fsolve(f,-2))
g=lambda x :1/x
r, infodict, ier, mesg = fsolve(g, -2, full_output=True)
print("r =", r)

result = f(r)
print("result=", result)

print(mesg)
#g has no root so the algorithm does not converge.

root for f is [0.73908513]
r = [-3.52047359e+83]
result= [3.52047359e+83]
The number of calls to function has reached maxfev = 400.


### Bisection method

This is basically a __binary search__ method based on the prerequisite that, if $f(a)>0$ and $f(b)<0$ (or viceversa) and $f$ is continuous, there must be a root of $f$ in the interval $(a,b)$.

In [44]:
def bisection_root(f,a,b,tol):
    #assume f(a)!=0
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")
        
    m=(a+b)/2
    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        #f(m) has same sign as f(a)
        return bisection_root(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        #f(m) has same sign as f(b)
        return bisection_root(f, a, m, tol)

In [45]:
bisection_root(lambda x:x**2-1+x,-1,1,0.001)

0.6181640625

### Newton-Raphson method

Let $f$ be smooth and $x_r$ an unknown root. Suppose $x_0$ is a good approximation of $x_r$, i.e. $f(x_0)$ is "small enough". You want to improve on estimation of $x_r$ by finding an $x_1$ which is closer to it than $x_0$.

If $x_0$ is close to $x_r$, you can take a linear approximation: $f(x_1)\sim f(x_0)+f'(x_0)(x_1-x_0)$. Imposing that $f(x_1)=0$, this becomes:
<center> $0=f(x_0)+f'(x_0)(x_1-x_0)\rightarrow x_1=x_0-\dfrac{f(x_0)}{f'(x_0)}$</center>
Basically the point $x_1$ is where the tangent to $f(x_0)$ intersecates the $x$ axis.

This method gets iterated until $f(x_n)$ is smaller than a given tolerance for some $n$. The steps are:
<center> $x_i=x_{i-1}-\dfrac{f(x_{i-1})}{f'(x_{i-1})}$</center>

In [57]:
f=lambda x: x**3+x**2-4*x -1
fp=lambda x: 3*x**2+2*x-4
def newton_raphson(x0,tol):
    if np.abs(f(x0))<tol:
        return x0
    x0=x0-f(x0)/fp(x0)
    return newton_raphson(x0,tol)

In [67]:
#solutions are -2.46,-0.23,1.69
print(newton_raphson(2,0.000001))
print(newton_raphson(-0.5,0.00001))
print(newton_raphson(-2,0.00001))

1.6996281482799458
-0.23912426707017212
-2.460504870019674


If $x_0$ is close to $x_r$, the Newton-Raphson converges much faster than the bisection method.

However:
- Without a good guess $x_0$ convergence speed is hard to determine.
- Even with a good guess $x_0$, we can have stability problems if $f'(x_0)\sim 0$, as the "Newton step" $f(x_0)/f'(x_0)$ will be large and will lead away from $x_0$.
- There's no precise way to tell to which root the algorithm will converge to.

__Example__ Consider the polynomial $f(x)=x^3−100 x^2−x+100$. This polynomial has a root at $x=1$ and $x=100$. We use the Newton-Raphson to find a root of $f$ starting at $x_0=0$.

In [73]:
def newton_raphson_generic(f,df,x0,tol):
    if abs(f(x0))<tol:
        return x0
    x0=x0-f(x0)/df(x0)
    return newton_raphson_generic(f,df,x0,tol)

poly=lambda x: x**3-100*x**2-x+100
dpoly=lambda x: 3*x**2-200*x-1

print(newton_raphson_generic(poly,dpoly,0,0.0001))

100.0


Even if we started near the root at $1$, the algorithm converged to the other root! This is because already at the very first step we had:
<center> $x_1=0-\frac{100}{-1}=100$ which is the second root of the polynomial.</center>

# Root finding in Python

The `scipy.optimize` package has a root finding function, `fsolve`. Apparently, it uses a modified Newton-Raphson algorithm (Powell's dog-leg method). It takes many arguments:
- `func` is the callable function
- `x0` is the starting point(s) (can be a `list`)
- `args` are extra arguments that you want to pass to `func`, by default they are `None`
- `fprime` if you want to pass to it the Jacobian matrix (i.e. partial derivatives matrix), otherwise it is automatically computed
- `xtol`: calculation will stop if difference between two consecutive iterations is lower than `xtol`
- `maxfev`: maximum number of iterations. By default it's `(len(x0)+1)*100`

  Note that only `func` and `x0` are required.

In [77]:
from scipy.optimize import fsolve

f = lambda x: x**3-100*x**2-x+100

fsolve(f, [2, 80]) #two starting points to find two different roots

array([  1., 100.])

## Problems

1) Write a function 𝑚𝑦_𝑓𝑖𝑥𝑒𝑑_𝑝𝑜𝑖𝑛𝑡(𝑓,𝑔,𝑡𝑜𝑙,𝑚𝑎𝑥𝑖𝑡𝑒𝑟), where 𝑓 and 𝑔 are function objects and 𝑡𝑜𝑙 and 𝑚𝑎𝑥_𝑖𝑡𝑒𝑟 are strictly positive scalars. The input argument, 𝑚𝑎𝑥_𝑖𝑡𝑒𝑟, is also an integer. The output argument, 𝑋, should be a scalar satisfying |𝑓(𝑋)−𝑔(𝑋)|<𝑡𝑜𝑙; that is, 𝑋 is a point that (almost) satisfies 𝑓(𝑋)=𝑔(𝑋). To find 𝑋, you should use the Bisection method with error metric, |𝐹(𝑚)|<𝑡𝑜𝑙. The function 𝑚𝑦_𝑓𝑖𝑥𝑒𝑑_𝑝𝑜𝑖𝑛𝑡 should “give up” after 𝑚𝑎𝑥_𝑖𝑡𝑒𝑟 number of iterations and return 𝑋=[] if this occurs.

In [98]:
def my_fixed_point(f,g,a,b,tol,maxiter):
    F=lambda x:f(x)-g(x)
    if maxiter>0:
        m=(a+b)/2
        if np.abs(F(m))<tol:
            return m
        elif np.sign(F(m))==np.sign(F(a)):
            return my_fixed_point(f,g,m,b,tol,maxiter-1)
        elif np.sign(F(m))==np.sign(F(b)):
            return my_fixed_point(f,g,a,m,tol,maxiter-1)
        else:
            return []
    else:
        return []

In [102]:
print(my_fixed_point(lambda x:x**2-1, lambda x:x-1, 0.5,3,0.001,10))
print(my_fixed_point(lambda x:x**2-1, lambda x:x-1, -1,0.8,0.001,10))
#The two fixed points are 0 and 1

1.00048828125
0.00019531250000002468


4) Write a function 𝑚𝑦_𝑏𝑖𝑠𝑒𝑐𝑡𝑖𝑜𝑛(𝑓,𝑎,𝑏,𝑡𝑜𝑙), that returns [𝑅,𝐸], where 𝑓 is a function object, 𝑎 and 𝑏 are scalars such that 𝑎<𝑏, and 𝑡𝑜𝑙 is a strictly positive scalar value. The function should return an array, 𝑅, where 𝑅[𝑖] is the estimation of the root of 𝑓 defined by (𝑎+𝑏)/2 for the 𝑖-th iteration of the bisection method. Remember to include the initial estimate. The function should also return an array, 𝐸, where 𝐸[𝑖] is the value of |𝑓(𝑅[𝑖])| for the 𝑖-th iteration of the bisection method. The function should terminate when 𝐸(𝑖)<𝑡𝑜𝑙. You may assume that sign(𝑓(𝑎))≠sign(𝑓(𝑏)).

In [113]:
def my_bisection(f,a,b,tol,R=[],E=[]):
    m=(a+b)/2
    m=(a+b)/2
    R.append(m)
    E.append(np.abs(f(m)))
    if np.abs(f(m))<tol:
        return R,E
    if np.sign(f(m))==np.sign(f(a)):
        return my_bisection(f,m,b,tol,R,E)
    elif np.sign(f(m))==np.sign(f(b)):
        return my_bisection(f,a,m,tol,R,E)
    else:
        return 'nope'

In [114]:
#test
f = lambda x: x**2 - 2
[R, E] = my_bisection(f, 0, 2, 1e-1)
#Out: R = [1, 1.5, 1.25, 1.375, 1.4375]
# E = [1, 0.25, 0.4375, 0.109375, 0.06640625]
print(R)
print(E)

[1.0, 1.5, 1.25, 1.375, 1.4375]
[1.0, 0.25, 0.4375, 0.109375, 0.06640625]


In [115]:
#test 2
f = lambda x: np.sin(x) - np.cos(x)
[R, E] = my_bisection(f, 0, 2, 1e-2)
#Out: R = [1, 0.5, 0.75, 0.875, 0.8125, 0.78125]
#    E = [0.30116867893975674, 0.39815702328616975, 0.05005010885048666, 0.12654664407270177, 0.038323093040207645, 0.005866372111545948]
print(R)
print(E)

[1.0, 1.5, 1.25, 1.375, 1.4375, 1.0, 0.5, 0.75, 0.875, 0.8125, 0.78125]
[1.0, 0.25, 0.4375, 0.109375, 0.06640625, 0.30116867893975674, 0.39815702328616975, 0.050050108850486774, 0.126546644072702, 0.038323093040207756, 0.005866372111545948]
